# West Virginia Mountaineers Football Team History

Loaded the table for the football team's statistics for the 106 years they have been playing.  Let's Go Mountaineers!

First import the modules needed.

In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html as lh

Get the data from the website.

In [2]:
r = requests.get("https://www.sports-reference.com/cfb/schools/west-virginia/")

In [3]:
#r.text    #Verified that data loaded.  Commented out for final submission. 

Inspect the data that was downloaded to look for the relationships and find what is needed.

In [4]:
html = lh.fromstring(r.text)

In [5]:
html

<Element html at 0x24545bdc7c8>

In [6]:
html      = lh.fromstring(r.text)
wvu_table = html.get_element_by_id("all_west-virginia")
tablebody = wvu_table.getchildren()[1].getchildren()[0].getchildren()[0].getchildren()[3]

In [7]:
wvu_table

<Element div at 0x24545bdcbd8>

Look for the table data

In [8]:
wvu_table.getchildren()[1].getchildren()[0].getchildren()[0].getchildren()[3]   # Here is the table body so I assigned this to the variable above

<Element tbody at 0x24545c121d8>

In [9]:
tablebody.getchildren()         # These are the table rows

[<Element tr at 0x24545c12228>,
 <Element tr at 0x24545c12318>,
 <Element tr at 0x24545c122c8>,
 <Element tr at 0x24545c12098>,
 <Element tr at 0x24545c12278>,
 <Element tr at 0x24545c123b8>,
 <Element tr at 0x24545c12408>,
 <Element tr at 0x24545c12458>,
 <Element tr at 0x24545c124a8>,
 <Element tr at 0x24545c124f8>,
 <Element tr at 0x24545c12548>,
 <Element tr at 0x24545c12598>,
 <Element tr at 0x24545c125e8>,
 <Element tr at 0x24545c12638>,
 <Element tr at 0x24545c12688>,
 <Element tr at 0x24545c126d8>,
 <Element tr at 0x24545c12728>,
 <Element tr at 0x24545c12778>,
 <Element tr at 0x24545c127c8>,
 <Element tr at 0x24545c12818>,
 <Element tr at 0x24545c12868>,
 <Element tr at 0x24545c128b8>,
 <Element tr at 0x24545c12908>,
 <Element tr at 0x24545c12958>,
 <Element tr at 0x24545c129a8>,
 <Element tr at 0x24545c129f8>,
 <Element tr at 0x24545c12a48>,
 <Element tr at 0x24545c12a98>,
 <Element tr at 0x24545c12ae8>,
 <Element tr at 0x24545c12b38>,
 <Element tr at 0x24545c12b88>,
 <Elemen

In [10]:
tablebody.getchildren().getchildren()    # I cannot get the table data from the table rows using the method in the videos 
                                         # because each table has a table header (th) as the first column before the td columns. I need a different method.
                                         # https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

AttributeError: 'list' object has no attribute 'getchildren'

In [11]:
tr_elements = html.xpath('//tr')            # Parse the data between the tr at the beginning and end of the row

In [12]:
[len(T) for T in tr_elements[:12]]          # Check to see that all rows have the same width.

[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

In [13]:
# Create and empty list
col=[]
i=0
# For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 Rk
2 Year
3 Conf
4 W
5 L
6 T
7 Pct
8 SRS
9 SOS
10 AP Pre
11 AP High
12 AP Post
13 Coach(es)
14 Bowl
15 Notes


In [14]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 15, the //tr data is not from our table 
    if len(T)!=15:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [15]:
[len(C) for (title,C) in col]         # All rows are the same length.  Would expect 106 
                                      # Have 5 extra rows.  Need to see why.

[111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111]

In [16]:
Dict={title:column for (title,column) in col}           # Create the dataframe
df=pd.DataFrame(Dict)

In [17]:
df.head()

,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
0,1,2018,Big 12,8,1,0,.889,17.25,2.25,17,6,,Dana Holgorsen (8-1),,
1,2,2017,Big 12,7,6,0,.538,4.84,3.07,22,22,,Dana Holgorsen (7-6),Heart of Dallas Bowl-L,
2,3,2016,Big 12,10,3,0,.769,9.57,2.03,,10,18,Dana Holgorsen (10-3),Russell Athletic Bowl-L,
3,4,2015,Big 12,8,5,0,.615,7.59,2.21,,23,,Dana Holgorsen (8-5),Cactus Bowl-W,
4,5,2014,Big 12,7,6,0,.538,8.07,4.45,,20,,Dana Holgorsen (7-6),Liberty Bowl-L,


In [18]:
df.describe()

,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
count,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111
unique,107,107,5,12,11,5,45,106,100,15,23,15,81,27,2
top,Rk,Year,Ind,8,4,0,.500,SRS,SOS,,,,,,
freq,5,5,60,19,27,78,7,5,5,88,65,87,8,70,106


In [19]:
df              # Looks like the page break headers from the website were brought in as records; 
                # confirm and then if so delete.

,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
0,1,2018,Big 12,8,1,0,.889,17.25,2.25,17,6,,Dana Holgorsen (8-1),,
1,2,2017,Big 12,7,6,0,.538,4.84,3.07,22,22,,Dana Holgorsen (7-6),Heart of Dallas Bowl-L,
2,3,2016,Big 12,10,3,0,.769,9.57,2.03,,10,18,Dana Holgorsen (10-3),Russell Athletic Bowl-L,
3,4,2015,Big 12,8,5,0,.615,7.59,2.21,,23,,Dana Holgorsen (8-5),Cactus Bowl-W,
4,5,2014,Big 12,7,6,0,.538,8.07,4.45,,20,,Dana Holgorsen (7-6),Liberty Bowl-L,
5,6,2013,Big 12,4,8,0,.333,-3.74,2.10,,,,Dana Holgorsen (4-8),,
6,7,2012,Big 12,7,6,0,.538,4.03,4.34,11,5,,Dana Holgorsen (7-6),Pinstripe Bowl-L,
7,8,2011,Big East,10,3,0,.769,8.65,0.96,24,11,17,Dana Holgorsen (10-3),Orange Bowl-W,
8,9,2010,Big East,9,4,0,.692,9.51,0.12,25,20,,Bill Stewart (9-4),Champs Sports Bowl-L,
9,10,2009,Big East,9,4,0,.692,7.48,2.48,,18,25,Bill Stewart (9-4),Gator Bowl-L,


In [20]:
bool_series = df.L == 'L'
duplicates = df[bool_series]
duplicates                                    # These are the duplicated header records

,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
20,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
41,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
62,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
83,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
104,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes


In [21]:
WVU_df = df[~bool_series]
WVU_df                                  # This is the dataframe without the duplicates.

,Rk,Year,Conf,W,L,T,Pct,SRS,SOS,AP Pre,AP High,AP Post,Coach(es),Bowl,Notes
0,1,2018,Big 12,8,1,0,.889,17.25,2.25,17,6,,Dana Holgorsen (8-1),,
1,2,2017,Big 12,7,6,0,.538,4.84,3.07,22,22,,Dana Holgorsen (7-6),Heart of Dallas Bowl-L,
2,3,2016,Big 12,10,3,0,.769,9.57,2.03,,10,18,Dana Holgorsen (10-3),Russell Athletic Bowl-L,
3,4,2015,Big 12,8,5,0,.615,7.59,2.21,,23,,Dana Holgorsen (8-5),Cactus Bowl-W,
4,5,2014,Big 12,7,6,0,.538,8.07,4.45,,20,,Dana Holgorsen (7-6),Liberty Bowl-L,
5,6,2013,Big 12,4,8,0,.333,-3.74,2.10,,,,Dana Holgorsen (4-8),,
6,7,2012,Big 12,7,6,0,.538,4.03,4.34,11,5,,Dana Holgorsen (7-6),Pinstripe Bowl-L,
7,8,2011,Big East,10,3,0,.769,8.65,0.96,24,11,17,Dana Holgorsen (10-3),Orange Bowl-W,
8,9,2010,Big East,9,4,0,.692,9.51,0.12,25,20,,Bill Stewart (9-4),Champs Sports Bowl-L,
9,10,2009,Big East,9,4,0,.692,7.48,2.48,,18,25,Bill Stewart (9-4),Gator Bowl-L,


In [22]:
type(WVU_df)

pandas.core.frame.DataFrame

In [23]:
for row in WVU_df.itertuples():                           #This is the data I want in the table in the database
    print(row[1:])

('1', 2018, 'Big 12', 8, 1, 0, '.889', '17.25', '2.25', 17, 6, '', 'Dana Holgorsen (8-1)', '', '')
('2', 2017, 'Big 12', 7, 6, 0, '.538', '4.84', '3.07', 22, 22, '', 'Dana Holgorsen (7-6)', 'Heart of Dallas Bowl-L', '')
('3', 2016, 'Big 12', 10, 3, 0, '.769', '9.57', '2.03', '', 10, 18, 'Dana Holgorsen (10-3)', 'Russell Athletic Bowl-L', '')
('4', 2015, 'Big 12', 8, 5, 0, '.615', '7.59', '2.21', '', 23, '', 'Dana Holgorsen (8-5)', 'Cactus Bowl-W', '')
('5', 2014, 'Big 12', 7, 6, 0, '.538', '8.07', '4.45', '', 20, '', 'Dana Holgorsen (7-6)', 'Liberty Bowl-L', '')
('6', 2013, 'Big 12', 4, 8, 0, '.333', '-3.74', '2.10', '', '', '', 'Dana Holgorsen (4-8)', '', '')
('7', 2012, 'Big 12', 7, 6, 0, '.538', '4.03', '4.34', 11, 5, '', 'Dana Holgorsen (7-6)', 'Pinstripe Bowl-L', '')
('8', 2011, 'Big East', 10, 3, 0, '.769', '8.65', '0.96', 24, 11, 17, 'Dana Holgorsen (10-3)', 'Orange Bowl-W', '')
('9', 2010, 'Big East', 9, 4, 0, '.692', '9.51', '0.12', 25, 20, '', 'Bill Stewart (9-4)', 'Champs Sp

Create the database and populate the table in the database with the data from the pandas dataframe.

In [24]:
conn = sqlite3.connect('WVU.db')
c = conn.cursor()

WVU_df.to_sql('WVUFinal', conn, if_exists='append', index=False)


C:\Users\sue\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [25]:
#c.execute('SELECT * FROM WVUFinal').fetchall()                  # This prints everything in the table
c.execute('SELECT Rk, Conf, "Coach(es)" FROM WVUFinal WHERE Conf="Big 12"').fetchall()     # This prints just the selected rows.

[('1', 'Big 12', 'Dana Holgorsen (8-1)'),
 ('2', 'Big 12', 'Dana Holgorsen (7-6)'),
 ('3', 'Big 12', 'Dana Holgorsen (10-3)'),
 ('4', 'Big 12', 'Dana Holgorsen (8-5)'),
 ('5', 'Big 12', 'Dana Holgorsen (7-6)'),
 ('6', 'Big 12', 'Dana Holgorsen (4-8)'),
 ('7', 'Big 12', 'Dana Holgorsen (7-6)')]

Close the session.

In [26]:
c.close()
conn.close()    
